# Notebook de clasificación de imágenes

## Conexión con Watson Machine Learning

In [1]:
api_key = '' # API Key de IBM Cloud
location = 'us-south' # Location del servicio de WML
space_id = '' # Espacio de despliegue
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}


In [2]:
#Se importan las librerías necesarias
!pip install tensorflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPool2D
from tensorflow.keras.models import Model

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 26.1 MB 15.0 MB/s eta 0:00:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scipy-1.5.0:
      Successfully uninstalled scipy-1.5.0


## Carga de datos y Normalización

In [3]:
#Cargamos el dataset Fashion MNIST
fashionMNIST = tf.keras.datasets.fashion_mnist
(xTrain, yTrain), (xTest, yTest) = fashionMNIST.load_data()
xTrain, xTest = (xTrain -127.5) / 127.5 , (xTest - 127.5) /127.5  # Normalizamos las imágenes
print("xTrainshape: ", xTrain.shape)

4423680/4422102 [==============================] - 0s 0us/step
xTrainshape:  (60000, 28, 28)


In [4]:
#Nuestras imágenes se pueden representar con matrices de 28 x 28. Tensorflow espera que tengan la forma H x W x C, por lo que tenemos que expandir las dimensiones
xTrain = np.expand_dims(xTrain, -1)
xTest = np.expand_dims(xTest, -1)
print (xTrain.shape)
#(Cantidad de imágenes, altura, ancho, color)

(60000, 28, 28, 1)


In [5]:
#cantidad de clases
K = len(set(yTrain))
print("Cantidad de clases: ", K)

Cantidad de clases:  10


## Arquitectura de la red

In [19]:
#Construcción del modelo. Leer el Readme para una explicación del funcionamiento de este.

i = Input(shape=xTrain[0].shape)
x = Conv2D(32, (3,3), activation= "relu")(i) #Capa de convolución
x = MaxPool2D(pool_size=(2, 2), padding="same")(x) #Capa de agrupamiento
x = Conv2D(64, (3,3), activation= "relu")(x)
x = MaxPool2D(pool_size=(2, 2), padding="same")(x) 
x = Conv2D(128, (3,3), activation= "relu")(x)
x = MaxPool2D(pool_size=(2, 2), padding="same")(x) 
x = Flatten()(x)
x = Dense(512, activation="relu")(x) #Capa completamente conectada 
x = Dense(64, activation="relu")(x)
x = Dense (K, activation = "softmax")(x)


model = Model(i,x)
model.summary()


Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 2, 2, 128)         0   

## Entrenamiento de la red

In [21]:
#Compile and Fit. Aquí definimos las formas de hacer el descenso de gradiente (utilizamos adam), la función con la cual 
#medimos el loss (sparse categorical crossentropy) y las métricas a optimizar (en nuestro caso el accuracy). 
#Si al lector le interesa comprender  qué significan estas funciones, puede buscar en la documentación de Keras 
#(https://keras.io/api/optimizers/adam/) 
#(https://keras.io/api/losses/probabilistic_losses/#sparse_categorical_crossentropy-function)
import math
model.compile(optimizer="adam", 
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

#Tenga en cuenta que la siguiente función puede tardar más de 1 hora en correr. Se deben hacer muchos cálculos en el 
#entrenamiento de nuestra red neuronal.
#Si se tiene un entorno con gpu o la siguiente función se termina de ejecutar rápidamente, se puede aumentar el numero de epochs 
#por ejemplo a 5 o a 10. Para verificar si la performance del modelo mejoró con el aumento de epochs (iteraciones),
#habría que fijarse en la validation_accuracy final de cada epoch. Si esta mejora a medida que aumentan los epochs,
#el modelo se ve beneficiado por el aumento de los mismos. En el caso de que esta (validation_accuracy) empeore y la accuracy 
#mejore, eso quiere decir que está ocurriendo un overfitting (el modelo aprende características de los datos de entrenamiento
#que no queremos que aprenda). En este caso, recomendaría bajar los epochs hasta que esto no ocurra.
r = model.fit(xTrain, yTrain, validation_data =(xTest, yTest), epochs = 1) 

Train on 60000 samples, validate on 10000 samples
   96/60000 [..............................] - ETA: 2:07:41 - loss: 2.2220 - accuracy: 0.2344

KeyboardInterrupt: 

In [ ]:
#plot accuracy
import matplotlib.pyplot as plt
plt.plot(r.history["accuracy"], label="acc")
plt.plot(r.history["val_accuracy"], label="val_acc")
plt.legend()

In [ ]:
#plot loss
plt.plot(r.history["loss"], label="loss")
plt.plot(r.history["val_loss"], label="val_loss")
plt.legend()


## Pruebas contra el modelo


In [ ]:
p_test = model.predict(xTest).argmax(axis=1)

In [ ]:
labels = """
T-shirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle boot
""".split()
print(labels)

In [ ]:
#Mostramos imágenes mal clasificadas

misclassifed_imgs = np.where(p_test !=yTest)[0]
i= np.random.choice(misclassifed_imgs)
plt.imshow(xTest[i].reshape(28,28), cmap="gray")
plt.title("True label: %s Predicted: %s"% (labels[yTest[i]], labels[p_test[i]]))

## Guardamos el modelo

In [ ]:
# Guardamos el modelo en el entorno previo a subirlo a Watson Machine learning
model.save("mejorModelo.h5")

## Subir a Watson ML

In [ ]:
#Dependencias
!pip install -U ibm-watson-machine-learning

In [ ]:
import os
from ibm_watson_machine_learning import APIClient

In [ ]:
# Revisamos si la configuración está OK
client = APIClient(wml_credentials)
client.spaces.list(limit=10)

In [ ]:
# Seleccionamos espacio de despliegue
client.set.default_space(space_id)

In [ ]:
if os.path.exists('tf_classification_model.tar.gz'):
    os.remove('tf_classification_model.tar.gz')

In [ ]:
!tar -cvf ./tf_classification_model.tar mejorModelo.h5 ModeloClasesGrandesKfold.h5

In [ ]:
!gzip tf_classification_model.tar

In [ ]:
model_path = 'tf_classification_model.tar.gz'

In [ ]:
!ls

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("tensorflow_2.1-py3.7") # UID de la versión del modelo en IBM Cloud
model_meta = {
    client.repository.ModelMetaNames.NAME: 'Primer Modelo Deep Learning',
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.1',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid

}
model_details = client.repository.store_model(model=model_path, meta_props=model_meta)

## Despliegue del Modelo

In [ ]:
model_uid = client.repository.get_model_uid(model_details)

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment Primer Modelo",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(model_uid, meta_props=metadata)